# (1) 라이브러리 불러오기

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spst
from statsmodels.graphics.mosaicplot import mosaic      #mosaic plot!
import joblib

#한글폰트 지정
import matplotlib.font_manager as fm

# 설치된 폰트
font_list = [font.name for font in fm.fontManager.ttflist]

# 맑은고딕이 있는지 확인
'Malgun Gothic' in font_list

plt.rcParams['font.family'] = 'Malgun Gothic'

# (2) 데이터 불러오기 
1. 전남 총 인구수 = '2024_전라남도_인구_추계.csv'
2. 전남 폐교 수 = '2024_전라남도교육청_폐교재산기본정보.csv'
3. 전국 학교 학생수 현황 = '2024_초등학교_전라남도.csv'

In [71]:
# 로컬 경로 (다운받아 사용하신다면 수정해서 사용하세요 )
path = ''

In [72]:
file = '2024_전라남도_인구_추계.csv'
data01= pd.read_csv(path + file)
# data01.head()

In [73]:
path = ''
file = '2024_전라남도교육청_폐교재산기본정보.csv'
data02= pd.read_csv(path + file)
# data02.head()

In [74]:
path = ''
file = '2024_전국초등학교.csv'
data03 = pd.read_csv(path + file)
# data03.head()

# (3) 데이터 전처리

In [75]:
# 데이터 복사 (원본 데이터를 지키도록 )
data_total_human= data01.copy()  # 총인구수 
data_close_school = data02.copy() # 폐교 학교 
data_total_school = data03.copy() # 전국국학교 

## 총 인구수 전처리 (결측치 처리리)
1. xlsx > csv 파일로 변환하는 과정에서 삭제된 행 (전부 NA로 구성) 이 표기되는 문제   
2. 중간에 인구수가 비어있는 경우 [-1 년도]~ [+1년도] 의 합 평균으로 대체 
3. 결측치가 있어서 float 으로 표기되는 문제 > int로 변환 

In [60]:
# data_total

In [76]:
# 전라남도 총 인구수 
# 1. 결측치 제거 (행이 전부 NA )
data_total_human = data_total_human.dropna(axis=0,how='all')
# data_total

In [77]:
# 2. 2015년부터 2040년까지의 결측치를 이전 년도와 이후 년도의 평균값으로 채우기
for year in range(2015, 2040):
    year_str = str(year)
    next_year_str = str(year + 1) if year + 1 <= 2040 else None
    prev_year_str = str(year - 1) if year - 1 >= 2015 else None
    
    for index, row in data_total_human.iterrows():
        if pd.isna(row[year_str]):  # 해당 연도의 값이 결측치일 경우
            prev_value = data_total_human.at[index, prev_year_str] if prev_year_str else None
            next_value = data_total_human.at[index, next_year_str] if next_year_str else None

            # 이전 년도와 이후 년도의 평균으로 결측치를 채움
            if prev_value is not None and next_value is not None:
                data_total_human.at[index, year_str] = (prev_value + next_value) / 2
            elif prev_value is not None:
                data_total_human.at[index, year_str] = prev_value
            elif next_value is not None:
                data_total_human.at[index, year_str] = next_value
                
# data_total.isna().sum()

In [78]:
cols_to_convert = [str(year) for year in range(2015, 2041)]  # 2015~2040 열 선택
data_total_human[cols_to_convert] = data_total_human[cols_to_convert].astype(int) # 정수로 변환 

# data_total.head()
# 결측치 수 확인
# data_total.isna().sum()결측치 수 확인-

## 폐교 데이터 전처리 
1. 삭제할 컬럼들을 제거 

In [79]:
#data_close_school.head() 


In [33]:
# 삭제할 컬럼 
drop_cols = ['시도교육청명','시도교육청코드','교육지원청코드','교육지원청명','시도코드','시도명','시군구코드','담당자 부서명','담당자 전화번호','소재지도로명주소','소재지지번주소','데이터기준일자']

data_close_school=data_close_school.drop(columns=drop_cols)

data_close_school

,시도교육청명,시군구명,폐교명,폐교연도,학교급구분명,활용현황구분명,건물연면적,대지
0,전라남도교육청,강진군,강진서초영파분교장,1991,초등학교,미활용,0,9250
1,전라남도교육청,강진군,강진북초등학교,2024,초등학교,미활용,2346,20534
2,전라남도교육청,고흥군,과역동초백일분교장,2007,초등학교,미활용,211,5056
3,전라남도교육청,고흥군,나로고등학교,2014,고등학교,미활용,3473,21377
4,전라남도교육청,고흥군,동강초매곡분교장,1997,초등학교,미활용,1130,13278
...,...,...,...,...,...,...,...,...
112,전라남도교육청,화순군,아산초노치분교장,1994,초등학교,미활용,106,3309
113,전라남도교육청,화순군,동면초옥호분교장,1995,초등학교,미활용,289,3465
114,전라남도교육청,화순군,아산초송방분교장,1996,초등학교,미활용,835,7100
115,전라남도교육청,화순군,화순초수만분교장,1997,초등학교,미활용,362,5014


## 작은학교 데이터 전처리 
1. 삭제할 컬럼 제거 
2. '학급수(계)'와 '학생수(계)'에서 괄호와 숫자를 제거 

In [80]:
# data_total_school

In [81]:
drop_cols = ['정보공시 학교코드','학교급코드','제외여부','제외사유']

data_total_school=data_total_school.drop(columns=drop_cols)

data_total_school

,시도교육청,교육지원청,지역,학교명,설립구분,1학년,2학년,3학년,4학년,5학년,6학년,특수학급,순회학급,학급수(계),학생수(계),학급당학생수
0,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 서초구,서울교육대학교부설초등학교,국립,4,4,4,5,5,5,1,0,28(1),611(5),21.8
1,서울특별시교육청,서울특별시중부교육지원청,서울특별시 종로구,서울대학교사범대학부설초등학교,국립,5,5,5,5,5,5,1,0,31(1),576(6),18.6
2,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,서울개원초등학교,공립,7,4,5,4,5,4,1,0,30(1),868(8),28.9
3,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,서울개일초등학교,공립,5,5,6,5,6,6,0,0,33(0),967(0),29.3
4,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,서울개포초등학교,공립,8,7,6,7,7,7,2,0,44(2),903(4),20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,월랑초등학교,공립,3,4,3,4,4,5,1,0,24(1),569(8),23.7
6298,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,이도초등학교,공립,6,6,8,7,7,9,1,0,44(1),1096(11),24.9
6299,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,도련초등학교,공립,6,7,8,8,8,8,2,0,47(2),1129(16),24.0
6300,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,삼화초등학교,공립,4,4,5,4,5,6,1,0,29(1),680(5),23.4


In [82]:
# '학급수(계)'와 '학생수(계)'에서 괄호와 숫자를 제거
data_total_school['학급수(계)'] = data_total_school['학급수(계)'].str.extract('(\d+)')  # 괄호 안 숫자 제거하고 숫자만 추출
data_total_school['학생수(계)'] = data_total_school['학생수(계)'].str.extract('(\d+)')  

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\User\AppData\Local\Temp\ipykernel_19624\2036548752.py:2: SyntaxWarning: invalid escape sequence '\d'
  data_total_school['학급수(계)'] = data_total_school['학급수(계)'].str.extract('(\d+)')  # 괄호 안 숫자 제거하고 숫자만 추출
C:\Users\User\AppData\Local\Temp\ipykernel_19624\2036548752.py:3: SyntaxWarning: invalid escape sequence '\d'
  data_total_school['학생수(계)'] = data_total_school['학생수(계)'].str.extract('(\d+)')


In [83]:
data_total_school

,시도교육청,교육지원청,지역,학교명,설립구분,1학년,2학년,3학년,4학년,5학년,6학년,특수학급,순회학급,학급수(계),학생수(계),학급당학생수
0,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 서초구,서울교육대학교부설초등학교,국립,4,4,4,5,5,5,1,0,28,611,21.8
1,서울특별시교육청,서울특별시중부교육지원청,서울특별시 종로구,서울대학교사범대학부설초등학교,국립,5,5,5,5,5,5,1,0,31,576,18.6
2,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,서울개원초등학교,공립,7,4,5,4,5,4,1,0,30,868,28.9
3,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,서울개일초등학교,공립,5,5,6,5,6,6,0,0,33,967,29.3
4,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,서울개포초등학교,공립,8,7,6,7,7,7,2,0,44,903,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,월랑초등학교,공립,3,4,3,4,4,5,1,0,24,569,23.7
6298,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,이도초등학교,공립,6,6,8,7,7,9,1,0,44,1096,24.9
6299,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,도련초등학교,공립,6,7,8,8,8,8,2,0,47,1129,24.0
6300,제주특별자치도교육청,제주시교육지원청,제주특별자치도 제주시,삼화초등학교,공립,4,4,5,4,5,6,1,0,29,680,23.4


## 신규 데이터프레임 생성 
1. 전국 학교 현황 데이터에서 , 시도교육청 ='전라남도교육청' 인 열들만 구성된 데이터 프레임 생성 

In [84]:
# '시도교육청'이 '전라남도교육청'인 행만 필터링
data_target_school = data_total_school[data_total_school['시도교육청'] == '전라남도교육청']
data_target_school

,시도교육청,교육지원청,지역,학교명,설립구분,1학년,2학년,3학년,4학년,5학년,6학년,특수학급,순회학급,학급수(계),학생수(계),학급당학생수
4722,전라남도교육청,전라남도목포교육지원청,전라남도 목포시,광주교육대학교목포부설초등학교,국립,3,3,3,3,3,3,1,0,19,394,20.7
4723,전라남도교육청,전라남도강진교육지원청,전라남도 강진군,강진동초등학교,공립,1,1,1,1,1,1,1,0,7,54,7.7
4724,전라남도교육청,전라남도강진교육지원청,전라남도 강진군,강진중앙초등학교,공립,5,5,6,5,4,4,1,0,30,608,20.3
4725,전라남도교육청,전라남도강진교육지원청,전라남도 강진군,계산초등학교,공립,1,1,1,1,1,1,1,0,7,52,7.4
4726,전라남도교육청,전라남도강진교육지원청,전라남도 강진군,군동초등학교,공립,1,1,1,1,1,1,1,0,7,30,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5165,전라남도교육청,전라남도순천교육지원청,전라남도 순천시,신대초등학교,공립,12,12,11,12,10,11,0,0,68,1849,27.2
5166,전라남도교육청,전라남도나주교육지원청,전라남도 나주시,빛누리초등학교,공립,10,9,12,9,9,10,2,0,61,1402,23.0
5167,전라남도교육청,전라남도무안교육지원청,전라남도 무안군,무안행복초등학교,공립,11,12,12,12,9,9,1,0,66,1636,24.8
5168,전라남도교육청,전라남도광양교육지원청,전라남도 광양시,광양와우초등학교,공립,3,3,2,2,2,2,0,0,14,303,21.6


사용할 데이터프레임(중간 보고)
data_total_human : 전남 인구수 추이(2015~ 2040), 나이대별
data_close_school: 전남 폐교 학교 정보()
data_total_school: 전국 학교 데이터
data_target_school: 전국 학교 데이터 중에서 '전라남도' 데이터  

### (3) 다변량 분석 함수

In [3]:
def categorical_numerical(df, feature, target) : # 범주형 , 숫자형 변수간 검정  (t-test, Anova) / 막대그래프 시각화 / 
    
    '''
    df = 데이터프레임
    feature = 구분 데이터프레임의 컬럼명
    target  = 목표 데이터프레임의 컬럼명
    '''
    
    sns.barplot(x = feature, y = target, data = df)
    plt.grid()
    plt.show()

    an_list = []
    for i in range(df[feature].nunique()) :
        an_list.append(df.loc[df[feature] == i, target])

    if len(an_list) == 2:
    # t-test
        t_stat, p_value = spst.ttest_ind(an_list[0], an_list[1])
        print(f"T-test 결과: t-statistic = {t_stat}, p-value = {p_value}")

    elif len(an_list) > 2:
    # ANOVA
        f_stat, p_value = spst.f_oneway(*an_list)
        print(f"ANOVA 결과: F-statistic = {f_stat}, p-value = {p_value}")

    else:
        print("비교할 그룹이 부족합니다.")
        
def categorical_categorical(df , feature , target): # 범주형 --> 범주형 간 검정 (카이제곱검정) / 모자이크플롯
    '''
    df = 데이터프레임
    feature = 구분 데이터프레임의 컬럼명
    target  = 목표 데이터프레임의 컬럼명
    '''
    mosaic(df, [ feature,target])
    #plt.axhline(1- df[target].mean(), color = 'r')
    plt.grid()
    plt.show()
    
    # 먼저 집계
    table = pd.crosstab(df[target], df[feature])
    print('교차표\n', table)
    print('=' * 100)

    # 카이제곱검정
    result = spst.chi2_contingency(table)
    print('카이제곱통계량', result[0])
    print('p-value', result[1])
    print('자유도', result[2])
    # print('기대빈도\n',result[3])
    
    
    
def numerical_numerical(df, feature , target): # 숫자형 , 숫자형 
    # 산점도 + 회귀분석 선 표기
    
    '''
    df = 데이터프레임
    feature = 구분 데이터프레임의 컬럼명
    target  = 목표 데이터프레임의 컬럼명
    '''
    sns.regplot(x=feature, y = target, data = df)
    plt.grid()
    plt.show()
    print('='*100)
    result = spst.pearsonr(df[feature], df[target])
    print(f'상관계수 : {result[0]}, p-value : {result[1]}')


def numerical_numerical_hitmap(df):
    '''
    df = 데이터프레임
    feature = 구분 데이터프레임의 컬럼명
    target  = 목표 데이터프레임의 컬럼명
    '''
    # 숫자형으로 구성된 데이터들 한해서만 상관분석 수행 
    temp = df.select_dtypes(include=['number'])
    plt.figure(figsize = (8, 8))
    # 한글 폰트 설정: 맑은 고딕
    plt.rcParams['font.family'] = 'Malgun Gothic'
    # 유니코드 마이너스 사용 설정
    plt.rcParams['axes.unicode_minus'] = False
    sns.heatmap(temp.corr(),
            annot = True,            # 숫자(상관계수) 표기 여부
            fmt = '.2f',             # 숫자 포멧 : 소수점 3자리까지 표기
            cmap = 'RdYlBu_r',       # 칼라맵
            vmin = -1, vmax = 1)     # 값의 최소, 최대값
    plt.show()